# Загрузка Pandas и очистка данных

In [1]:
import re
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import collections
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('main_task.xls')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [4]:
# Подготавливаем код для сортировки данных
pattern = re.compile('\d\d\D\d\d\D\d\d\d\d')
# Создаем новый столбец и вытаскиваем данные со старого
data['time_reviews'] = data['Reviews'].apply(lambda x: pattern.findall(x))
data[data.time_reviews.apply(lambda x: len(x)) ==
     3].time_reviews.iloc[0].pop(2)  # Единственная строка в которой присуствует лишние данные
data[data.time_reviews.apply(lambda x: len(x)) == 0].time_reviews.apply(
    lambda x: x.append('01/01/2017'))  # Подводим данные к единообразию для удобства работы с ними
data[data.time_reviews.apply(lambda x: len(x)) == 1].time_reviews.apply(
    lambda x: x.append(x[0]))  # Теперь все данные имеют одинаковое строение
data['time_reviews1'] = data.time_reviews.apply(lambda x: x[0])
data['time_reviews2'] = data.time_reviews.apply(lambda x: x[1])
data.time_reviews1 = pd.to_datetime(data.time_reviews1)
data.time_reviews2 = pd.to_datetime(data.time_reviews2)
# Получилось два новых стобика в которых отображается время двух отзывов, в данных в которых отсуствовали даты отзывов я специально поставил одинаковые числа
data['day'] = data.time_reviews1-data.time_reviews2
data.day = data.day.apply(lambda x: x.days)
data.day = pd.to_numeric(data.day)
# Получили новый столбец с данными о количестве дней прошедших между отзывами

In [5]:
# Приводим данные в порядок меняем числовое значение nan на строчное
data['Cuisine Style'] = data['Cuisine Style'].astype(
    object).replace(np.nan, 'None')
data['lenStyle'] = data['Cuisine Style'].apply(lambda x: x.split(','))
# Считаем количество стилей кухни предоставленной посетителям в каждом ресторане
data['lenStyle'] = data['lenStyle'].apply(lambda x: len(x))

In [6]:
data['Price Range'] = data['Price Range'].fillna(2)
# Заполняем пропуски средним значением
data['Price Range'][data['Price Range'] == '$$ - $$$'] = data['Price Range'][data['Price Range']
                                                                             == '$$ - $$$'].apply(lambda x: x.replace('$$ - $$$', '2'))
data['Price Range'][data['Price Range'] == '$'] = data['Price Range'][data['Price Range']
                                                                      == '$'].apply(lambda x: x.replace('$', '1'))
data['Price Range'][data['Price Range'] == '$$$$'] = data['Price Range'][data['Price Range']
                                                                         == '$$$$'].apply(lambda x: x.replace('$$$$', '3'))
# Так как ценовой диапазон меню каждого ресторана это ощютимый признак распределяем все на три категории
data['Price Range'] = pd.to_numeric(data['Price Range'])

C:\Users\Mudriy\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Mudriy\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Mudriy\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
data['Number of Reviews'] = data['Number of Reviews'].fillna(np.mean(
    data['Number of Reviews']))  # Заполняем пропуски средним значением всего столбца

In [8]:
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: x.replace("'", ""))
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: x.replace("[", ""))
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: x.replace("]", ""))
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: x.replace(" ", ""))
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: x.split(','))  # Чистим данные для удобства работы
a = data['Cuisine Style'].tolist()
b = [j for x in a for j in x]
c = collections.Counter()
for x in b:
    c[x] += 1  # Считаем количество каждого стиля кухни в датасете

def find_style(cell):
    if style in cell:
        return 1
    else:
        return 0  # Готовим функцию для заполнения значений в новых столбцах

# Методом проб и ошибок отбираем 50 стилей(лучшее EMA)
top_style = list(c)[:51]
for style in top_style:
    data[style] = data['Cuisine Style'].apply(
        find_style)  # Создаем 50 новых столбцов-стилей
no_top_style = list(c)[51:]

def non_top_style(x):
    for y in no_top_style:
        if y in x:
            return 1
            break
        else:
            return 0


data['other'] = data['Cuisine Style'].apply(
    non_top_style)  # Создаем столбец для невошедших стилей

In [9]:
# dummy переменные для столбца city
city = pd.get_dummies(data.City, drop_first=True)

In [10]:
# приводим данные для удобства в работе
data.Reviews = data.Reviews.apply(lambda x: x.lower())
very_bad = ['fab', 'lack', "don\'t", "hasn\'t",
            'nothing', 'non', 'disappoint', 'but']
bad = ['not bad', 'seriously', 'nice', 'standart', 'break']
norm = ['traditional', 'relaxing', 'quick', 'not more', 'fresh']
good = ['good', 'real', 'fast', 'cool', 'love', 'cosy']
very_good = ['unique', 'best', 'amazing', 'excellent',
             'great', 'perfect', 'wonderful', 'not miss', 'super']
zero = ['[]']  # Создаем списки ключевых слов в отзывах клиентов ресторана


def analitic_txt(x):
    for y in zero:
        if y in x:
            return 0
            break
    for q in very_good:
        if q in x:
            return 5
            break
    for w in good:
        if w in x:
            return 4
            break
    for t in norm:
        if t in x:
            return 3
            break
    for h in bad:
        if h in x:
            return 2
            break
    for k in very_bad:
        if k in x:
            return 1
            break  # Создаем функцию для оценки коментариев клиентов по ключевым словам


data['analitic_txt'] = data.Reviews.apply(analitic_txt)
# Заполняем пропуски таблицы в которой расположенны не отсортированные значения, более лучший балл ЕМА дает значение 0
data['analitic_txt'] = data['analitic_txt'].fillna(0)

In [11]:
# Соединяем dummy переменные с основной таблицей
data_all = pd.concat([data, city], axis=1)

In [12]:
correlation = data_all.corr()
plt.figure(figsize=(160, 160))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [13]:
a=data_all

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [14]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = a.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 'Rating', 'time_reviews', 'time_reviews1', 'time_reviews2'], axis = 1)
y = a['Rating']

In [15]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [16]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [17]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [18]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [19]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2174125
